# OMI 01

In [4]:
### Package Import
import sys
import os
base_dir = os.environ['GEMS_HOME']
project_path = os.path.join(base_dir, 'python-refactor')
sys.path.insert(0, project_path)
from Code.utils import matlab

import numpy as np
import glob
import time

In [5]:
### Setting path
data_base_dir = os.path.join(project_path, 'Data')
path_read = os.path.join(data_base_dir, 'Raw', 'OMI') 
path_write = os.path.join(data_base_dir, 'Preprocessed_raw', 'OMI')

In [6]:
YEARS = [2016]

In [4]:
def create_L3(year, read_path, flist, write_path, fname, data_type):
    doy_000 = matlab.datenum(f'{year}0000')
    if data_type == 'OMDOAO3e':
        list_doy = [matlab.datenum(f'{x[21:25]}{x[26:30]}')-doy_000 for x in flist]
    else:
        list_doy = [matlab.datenum(f'{x[19:23]}{x[24:28]}')-doy_000 for x in flist]
    size = (1036800, days)
    
    # Create data_yr
    
    with h5py.File(os.path.join(write_path, fname), 'w') as f:
        data_yr = f.create_dataset('data_yr', shape=size, compression="gzip",
                                   dtype=np.float64, fillvalue=-9999, chunks=True)
        for i, read_fname in enumerate(flist):
            print (f'Reading {data_type}_{year}_{list_doy[i]:03d}')
            #print (f"Reading {read_fname}")
            if data_type == 'OMNO2d':
                data = matlab.h5read(os.path.join(read_path, read_fname), 
                                 '/HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/ColumnAmountNO2TropCloudScreened')
                data = np.float64(data); 
                data[data<=-1.2676506e+30] = np.nan # Assign NaN value to pixel that is out of valid range
                data = data * 3.7216e-17   
            
            elif data_type in ['OMSO2e', 'OMDOAO3e']:
                if data_type == 'OMSO2e':
                    data = matlab.h5read(os.path.join(read_path, read_fname), 
                                          '/HDFEOS/GRIDS/OMI Total Column Amount SO2/Data Fields/ColumnAmountSO2_PBL')
                    rcf = matlab.h5read(os.path.join(read_path, read_fname), 
                                         '/HDFEOS/GRIDS/OMI Total Column Amount SO2/Data Fields/RadiativeCloudFraction')
                    sza = matlab.h5read(os.path.join(read_path, read_fname), 
                                         '/HDFEOS/GRIDS/OMI Total Column Amount SO2/Data Fields/SolarZenithAngle')
                elif data_type == 'OMDOAO3e':
                    data = matlab.h5read(os.path.join(read_path, read_fname), 
                                          '/HDFEOS/GRIDS/ColumnAmountO3/Data Fields/ColumnAmountO3')
                    rcf = matlab.h5read(os.path.join(read_path, read_fname), 
                                        '/HDFEOS/GRIDS/ColumnAmountO3/Data Fields/CloudFraction')
                    sza = matlab.h5read(os.path.join(read_path, read_fname), 
                                         '/HDFEOS/GRIDS/ColumnAmountO3/Data Fields/SolarZenithAngle')
                    
                data = np.float64(data); data[data<=-1.2676506e+30] = np.nan # Assign NaN value to pixel that is out of valid range
                rcf = np.float64(rcf); rcf[rcf<=-1.2676506e+30] = np.nan
                sza = np.float64(sza); sza[sza<=-1.2676506e+30] = np.nan

                # Filtering SO2_VCD with the condition of radiative cloud fraction and
                # solar zenith angle    
                data[rcf>=0.3]=np.nan
                data[sza>=78]=np.nan
            
            data[np.isnan(data)] = -9999 # trick. data_yr(isnan(data_yr))=-9999;
            data_yr[:, list_doy[i]-1] = data.flatten() # Put daily data as column vector
            # header = [f'd{x:03d}' for x in range(1, days+1)] # Not used after
    print (f"Created {fname}")

In [9]:
for yr in YEARS:
    tStart = time.time()
    doy_000 = matlab.datenum(f'{yr}0000')
    file_list = glob.glob(os.path.join(path_read, 'L3', 'OMNO2d', str(yr), '*.he5'))
    file_list.sort()
    
    if yr%4==0: days = 366
    else: days = 365
        
    data_yr = np.ones((1036800, days))*np.nan
    for read_fname in file_list:
        temp = os.path.basename(read_fname)
        doy = matlab.datenum(temp[19:23]+temp[24:28])-doy_000
        print (f'Reading {yr}_{doy:03d}')
        data = matlab.h5read(read_fname, 
                         '/HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/ColumnAmountNO2TropCloudScreened')
        data = np.float64(data.T); # 720X1440 
        data[data<=-1.2676506e+30] = np.nan # Assign NaN value to pixel that is out of valid range
        data = data * 3.7216e-17   
        data_yr[:, doy-1] = data.ravel(order='F')
    out_fname = os.path.join(path_write, f'OMNO2d_{yr}_DU.mat')
    matlab.check_make_dir(os.path.dirname(out_fname))
    data_yr[np.isnan(data_yr)] = -9999 
    matlab.savemat(out_fname, {'data_yr':data_yr})
    tElapsed = time.time() - tStart
    print (f'{tElapsed} second')
print ('==========================================================')

Reading 2016_001
Reading 2016_002
Reading 2016_003
Reading 2016_004
Reading 2016_005
Reading 2016_006
Reading 2016_007
Reading 2016_008
Reading 2016_009
Reading 2016_010
Reading 2016_011
Reading 2016_012
Reading 2016_013
Reading 2016_014
Reading 2016_015
Reading 2016_016
Reading 2016_017
Reading 2016_018
Reading 2016_019
Reading 2016_020
Reading 2016_021
Reading 2016_022
Reading 2016_023
Reading 2016_024
Reading 2016_025
Reading 2016_026
Reading 2016_027
Reading 2016_028
Reading 2016_029
Reading 2016_030
Reading 2016_031
Reading 2016_032
Reading 2016_033
Reading 2016_034
Reading 2016_035
Reading 2016_036
Reading 2016_037
Reading 2016_038
Reading 2016_039
Reading 2016_040
Reading 2016_041
Reading 2016_042
Reading 2016_043
Reading 2016_044
Reading 2016_045
Reading 2016_046
Reading 2016_047
Reading 2016_048
Reading 2016_049
Reading 2016_050
Reading 2016_051
Reading 2016_052
Reading 2016_053
Reading 2016_054
Reading 2016_055
Reading 2016_056
Reading 2016_057
Reading 2016_058
Reading 2016_0

In [36]:
print ('OMSO2e')
for yr in YEARS:
    tStart = time.time()
    doy_000 = matlab.datenum(f'{yr}0000')
    file_list = glob.glob(os.path.join(path_read, 'L3', 'OMSO2e', str(yr), '*.he5'))
    file_list.sort()
    
    if yr%4==0: days = 366
    else: days = 365
        
    data_yr = np.ones((1036800, days))*np.nan        
    for read_fname in file_list:
        temp = os.path.basename(read_fname)
        doy = matlab.datenum(temp[19:23]+temp[24:28])-doy_000
        print (f'Reading {yr}_{doy:03d}')
        data = matlab.h5read(read_fname,
                              '/HDFEOS/GRIDS/OMI Total Column Amount SO2/Data Fields/ColumnAmountSO2_PBL')
        rcf = matlab.h5read(read_fname,
                             '/HDFEOS/GRIDS/OMI Total Column Amount SO2/Data Fields/RadiativeCloudFraction')
        sza = matlab.h5read(read_fname,
                             '/HDFEOS/GRIDS/OMI Total Column Amount SO2/Data Fields/SolarZenithAngle')
        data = np.float64(data.T); data[data<=-1.2676506e+30] = np.nan 
        rcf = np.float64(rcf.T); rcf[rcf<=-1.2676506e+30] = np.nan
        sza = np.float64(sza.T); sza[sza<=-1.2676506e+30] = np.nan
        data[rcf>=0.3]=np.nan
        data[sza>=78]=np.nan
        data_yr[:, doy-1] = data.ravel(order='F')
    out_fname = os.path.join(path_write, f'OMSO2d_{yr}.mat')
    matlab.check_make_dir(os.path.dirname(out_fname))
    data_yr[np.isnan(data_yr)] = -9999 
    matlab.savemat(out_fname, {'data_yr':data_yr})
    tElapsed = time.time() - tStart
    print (f'{tElapsed} second')
print ('==========================================================')    

OMSO2e
Reading 2016_001
Reading 2016_002
Reading 2016_003
Reading 2016_004
Reading 2016_005


/home/cogito/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal
/home/cogito/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater_equal


Reading 2016_006
Reading 2016_007
Reading 2016_008
Reading 2016_009
Reading 2016_010
Reading 2016_011
Reading 2016_012
Reading 2016_013
Reading 2016_014
Reading 2016_015
Reading 2016_016
Reading 2016_017
Reading 2016_018
Reading 2016_019
Reading 2016_020
Reading 2016_021
Reading 2016_022
Reading 2016_023
Reading 2016_024
Reading 2016_025
Reading 2016_026
Reading 2016_027
Reading 2016_028
Reading 2016_029
Reading 2016_030
Reading 2016_031
Reading 2016_032
Reading 2016_033
Reading 2016_034
Reading 2016_035
Reading 2016_036
Reading 2016_037
Reading 2016_038
Reading 2016_039
Reading 2016_040
Reading 2016_041
Reading 2016_042
Reading 2016_043
Reading 2016_044
Reading 2016_045
Reading 2016_046
Reading 2016_047
Reading 2016_048
Reading 2016_049
Reading 2016_050
Reading 2016_051
Reading 2016_052
Reading 2016_053
Reading 2016_054
Reading 2016_055
Reading 2016_056
Reading 2016_057
Reading 2016_058
Reading 2016_059
Reading 2016_060
Reading 2016_061
Reading 2016_062
Reading 2016_063
Reading 2016_0

In [7]:
### OMDOAO3e
print ('OMDOAO3e')
for yr in YEARS:
    tStart = time.time()
    doy_000 = matlab.datenum(f'{yr}0000')
    file_list = glob.glob(os.path.join(path_read, 'L3', 'OMDOAO3e', str(yr), '*.he5'))
    file_list.sort()
    
    if yr%4==0: days = 366
    else: days = 365
        
    data_yr = np.ones((1036800, days))*np.nan        
    for read_fname in file_list:
        temp = os.path.basename(read_fname)
        doy = matlab.datenum(temp[21:25]+temp[26:30])-doy_000
        print (f'Reading {yr}_{doy:03d}')
        data = matlab.h5read(read_fname,
                              '/HDFEOS/GRIDS/ColumnAmountO3/Data Fields/ColumnAmountO3')
        rcf = matlab.h5read(read_fname,
                             '/HDFEOS/GRIDS/ColumnAmountO3/Data Fields/CloudFraction')
        sza = matlab.h5read(read_fname,
                             '/HDFEOS/GRIDS/ColumnAmountO3/Data Fields/SolarZenithAngle')
        data = np.float64(data.T); data[data<=-1.2676506e+30] = np.nan 
        rcf = np.float64(rcf.T); rcf[rcf<=-1.2676506e+30] = np.nan
        sza = np.float64(sza.T); sza[sza<=-1.2676506e+30] = np.nan
        data[rcf>=0.3]=np.nan
        data[sza>=78]=np.nan
        data_yr[:, doy-1] = data.ravel(order='F')
    out_fname = os.path.join(path_write, f'OMDOAO3e_m_{yr}.mat')
    matlab.check_make_dir(os.path.dirname(out_fname))
    data_yr[np.isnan(data_yr)] = -9999 
    matlab.savemat(out_fname, {'data_yr':data_yr})
    tElapsed = time.time() - tStart
    print (f'{tElapsed} second')
print ('==========================================================')

OMDOAO3e
Reading 2016_001
Reading 2016_002
Reading 2016_003
Reading 2016_004


/home/cogito/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater_equal
/home/cogito/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in greater_equal


Reading 2016_005
Reading 2016_006
Reading 2016_007
Reading 2016_008
Reading 2016_009
Reading 2016_010
Reading 2016_011
Reading 2016_012
Reading 2016_013
Reading 2016_014
Reading 2016_015
Reading 2016_016
Reading 2016_017
Reading 2016_018
Reading 2016_019
Reading 2016_020
Reading 2016_021
Reading 2016_022
Reading 2016_023
Reading 2016_024
Reading 2016_025
Reading 2016_026
Reading 2016_027
Reading 2016_028
Reading 2016_029
Reading 2016_030
Reading 2016_031
Reading 2016_032
Reading 2016_033


KeyboardInterrupt: 

In [8]:
### OMHCHOG
print ('OMHCHOG')
for yr in YEARS:
    tStart = time.time()
    doy_000 = matlab.datenum(f'{yr}0000')
    file_list = glob.glob(os.path.join(path_read, 'L2', 'OMHCHOG', str(yr), '*.he5'))
    file_list.sort()
    
    if yr%4==0: days = 366
    else: days = 365

    data_yr = np.ones((1036800, days))*np.nan        
    for read_fname in file_list:
        temp = os.path.basename(read_fname)
        doy = matlab.datenum(temp[21:25]+temp[26:30])-doy_000
        print (f'Reading {yr}_{doy:03d}')
        data = matlab.h5read(read_fname,
                              '/HDFEOS/GRIDS/OMI Total Column Amount HCHO/Data Fields/ColumnAmountDestriped')
        data = np.float64(data); data = np.transpose(data, (1,0,2));
        data[data<=-1.0e+30] = np.nan
        VCD = data * 3.7216e-17
        
        QA = matlab.h5read(read_fname,
                              '/HDFEOS/GRIDS/OMI Total Column Amount HCHO/Data Fields/MainDataQualityFlag')
        QA = np.float64(QA); QA = np.transpose(QA, (1,0,2));
        QA[QA<=-30000] = np.nan
        
        sza = matlab.h5read(read_fname,
                                '/HDFEOS/GRIDS/OMI Total Column Amount HCHO/Data Fields/SolarZenithAngle')
        sza = np.float64(sza); sza = np.transpose(sza, (1,0,2));
        sza[sza<=-1.0e+30] = np.nan
        
        VCD_filtered = np.zeros(VCD.shape)
        for k in range(15):
            VCD_temp = VCD[:,:,k]
            QA_temp = QA[:,:,k]
            sza_temp = sza[:,:,k]

            VCD_temp[QA_temp!=0] = np.nan #  Good columns (0) / Suspect columns (1) / Bad columns (2) / missing (<0)
            VCD_temp[sza_temp>88] = np.nan
            VCD_filtered[:,:,k] = VCD_temp
            del VCD_temp, QA_temp, sza_temp
        VCD_avg = np.nanmean(VCD_filtered, axis=2)
        data_yr[:, doy-1] = VCD_avg.ravel(order='F')
        del data, VCD, QA, sza
    out_fname = os.path.join(path_write, f'OMHCHOG_{yr}.mat')
    data_yr[np.isnan(data_yr)] = -9999 
    matlab.savemat(out_fname, {'data_yr':data_yr})
    tElapsed = time.time() - tStart
    print (f'{tElapsed} second')
print ('==========================================================')

OMHCHOG
Reading 2016_001


/home/cogito/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in greater
/home/cogito/.local/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice


Reading 2016_002
Reading 2016_003
Reading 2016_004
Reading 2016_005
Reading 2016_006
Reading 2016_007
Reading 2016_008
Reading 2016_009
Reading 2016_010
Reading 2016_011
Reading 2016_012
Reading 2016_013
Reading 2016_014
Reading 2016_015
Reading 2016_016
Reading 2016_017
Reading 2016_018
Reading 2016_019
Reading 2016_020
Reading 2016_021
Reading 2016_022
Reading 2016_023
Reading 2016_024
Reading 2016_025
Reading 2016_026
Reading 2016_027
Reading 2016_028
Reading 2016_029
Reading 2016_030
Reading 2016_031
Reading 2016_032
Reading 2016_033
Reading 2016_034
Reading 2016_035
Reading 2016_036
Reading 2016_037
Reading 2016_038
Reading 2016_039
Reading 2016_040
Reading 2016_041
Reading 2016_042
Reading 2016_043
Reading 2016_044
Reading 2016_045
Reading 2016_046
Reading 2016_047
Reading 2016_048
Reading 2016_049
Reading 2016_050
Reading 2016_051
Reading 2016_052
Reading 2016_053
Reading 2016_054
Reading 2016_055
Reading 2016_056
Reading 2016_057
Reading 2016_058
Reading 2016_059
Reading 2016_0

In [9]:
mat = matlab.loadmat(out_fname)

In [12]:
mat2 = matlab.loadmat(os.path.join(os.path.dirname(out_fname), f'OMHCHOG_{yr}_matlab.mat'))

In [13]:
print (mat['data_yr'].shape)

print (mat2['data_yr'].shape)

np.all(mat['data_yr']==mat2['data_yr'])

(1036800, 366)
(1036800, 366)


False

In [14]:
mat['data_yr']

array([[ 6.15924806e-02, -9.99900000e+03, -9.99900000e+03, ...,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
        -9.99900000e+03, -3.57273606e-03, -9.99900000e+03],
       ...,
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03]])

In [30]:
np.all(np.isclose(mat['data_yr'], mat2['data_yr']))

True

In [19]:
np.where(mat['data_yr']!=mat2['data_yr'])

(array([     16,      18,      20, ..., 1036772, 1036782, 1036783]),
 array([ 37, 304,  51, ..., 224, 185, 127]))

In [26]:
mat['data_yr'][16, 37:47]

array([ 6.24298402e-02,  6.88495982e-03, -9.99900000e+03, -9.99900000e+03,
       -9.99900000e+03,  9.19607327e-02,  5.21023983e-04, -9.99900000e+03,
       -9.99900000e+03, -9.99900000e+03])

In [24]:
mat2['data_yr'][16, 37:47]

array([ 6.24298402e-02,  6.88495982e-03, -9.99900000e+03, -9.99900000e+03,
       -9.99900000e+03,  9.19607327e-02,  5.21023983e-04, -9.99900000e+03,
       -9.99900000e+03, -9.99900000e+03])

# OMI 02

In [1]:
### Package Import
import sys
import os
base_dir = os.environ['GEMS_HOME']
project_path = os.path.join(base_dir, 'python-refactor')
sys.path.insert(0, project_path)
from Code.utils import matlab

In [2]:
import numpy as np
from numba import njit

In [3]:
### Setting path
data_base_dir = os.path.join(project_path, 'Data')
path_read = os.path.join(data_base_dir, 'Preprocessed_raw', 'OMI') 

In [4]:
YEARS = [2016] #, 2018, 2019
pname_list = ['OMNO2d','OMSO2e_m','OMDOAO3e_m','OMHCHOG']

mask = np.zeros((720, 1440))
mask[340:552, 1020:1308] = 1
mask = mask.ravel(order='F')

In [ ]:
for pname in pname_list:
    print (pname)
    ### Load data
    YEARS = [2016]
    def read_and_mask(yr):
        if os.path.isfile(os.path.join(path_read, f'{pname}_{yr}_DU.mat')):
            data_yr = matlab.loadmat(os.path.join(path_read, f'{pname}_{yr}_DU.mat'))['data_yr']
        elif os.path.isfile(os.path.join(path_read, f'{pname}_trop_CS_{yr}_DU.mat')):
            data_yr = matlab.loadmat(os.path.join(path_read, f'{pname}_{yr}_DU.mat'))['data_yr']
        elif os.path.isfile(os.path.join(path_read, f'{pname}_{yr}.mat')):
            data_yr = matlab.loadmat(os.path.join(path_read, f'{pname}_{yr}.mat'))['data_yr']
        else:
            raise ValueError(f'{pname}_{yr}.mat does not exists though variation tried.')
        data_subset = data_yr[mask==1, :]
        return data_subset
    data = np.concatenate([read_and_mask(yr) for yr in YEARS], axis=1)  # hstack
    data[data==-9999] = np.nan
    sigma = 1
    
    @njit(error_model='numpy')
    def calculate(data):
        data_conv = np.full(data.shape, np.nan)
        for k in range(data.shape[0]):
            for t in range(5478): # for 15 years
                t_ = t+1
                w_sum = 0; x_sum = 0;
                for n in range(5478): # for 15 years
                    n_ = n+1
                    if np.isnan(data[k,n])==0:
                        w = np.exp((-(t_-n_)**2)/(2*sigma**2)) #  calcuate weights using t,n(days) with sigma # (k+1)-(n-1)=k-n
                        x = data[k, n]*w
                        w_sum += w
                        x_sum += x
                data_conv[k,t] = x_sum/w_sum
                """
                try:
                    data_conv[k,t] = x_sum/w_sum
                except ZeroDivisionError:
                    data_conv[k,t] = np.nan
                """
            print (k)
        return data_conv
    data_conv = calculate(data)
    matlab.savemat(os.path.join(path_read, f'tempConv_{pname}_sigma{sigma}_2005_2019.mat'), 
                   {'data_conv':data_conv, 'data':data})
    #matlab.savemat(os.path.join(path_read, f'tempConv_{pname}_trop_CS_sigma{sigma}_2005_2019.mat'),
    #                {'data_conv':data_conv, 'data':data})
    break

OMNO2d
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234


In [5]:
for pname in pname_list:
    print (pname)
    ### Load data
    YEARS = [2016]
    def read_and_mask(yr):
        if os.path.isfile(os.path.join(path_read, f'{pname}_{yr}_DU.mat')):
            data_yr = matlab.loadmat(os.path.join(path_read, f'{pname}_{yr}_DU.mat'))['data_yr']
        elif os.path.isfile(os.path.join(path_read, f'{pname}_trop_CS_{yr}_DU.mat')):
            data_yr = matlab.loadmat(os.path.join(path_read, f'{pname}_{yr}_DU.mat'))['data_yr']
        elif os.path.isfile(os.path.join(path_read, f'{pname}_{yr}.mat')):
            data_yr = matlab.loadmat(os.path.join(path_read, f'{pname}_{yr}.mat'))['data_yr']
        else:
            raise ValueError(f'{pname}_{yr}.mat does not exists though variation tried.')
        data_subset = data_yr[mask==1, :]
        return data_subset
    data = np.concatenate([read_and_mask(yr) for yr in YEARS], axis=1)  # hstack
    data[data==-9999] = np.nan
    sigma = 1
    
    
    def calculate(data):
        data_conv = np.full(data.shape, np.nan)
        for k in range(data.shape[0]):
            for t in range(5478): # for 15 years
                t_ = t+1
                w_sum = 0; x_sum = 0;
                for n in range(5478): # for 15 years
                    n_ = n+1
                    if np.isnan(data[k,n])==0:
                        w = np.exp((-(t_-n_)**2)/(2*sigma**2)) #  calcuate weights using t,n(days) with sigma # (k+1)-(n-1)=k-n
                        x = data[k, n]*w
                        w_sum += w
                        x_sum += x
                data_conv[k,t] = x_sum/w_sum
                """
                try:
                    data_conv[k,t] = x_sum/w_sum
                except ZeroDivisionError:
                    data_conv[k,t] = np.nan
                """
            print (k)
        return data_conv
    data_conv = calculate(data)
    matlab.savemat(os.path.join(path_read, f'tempConv_{pname}_sigma{sigma}_2005_2019.mat'), 
                   {'data_conv':data_conv, 'data':data})
    #matlab.savemat(os.path.join(path_read, f'tempConv_{pname}_trop_CS_sigma{sigma}_2005_2019.mat'),
    #                {'data_conv':data_conv, 'data':data})
    break

OMNO2d


IndexError: index 366 is out of bounds for axis 1 with size 366

In [9]:
mat = matlab.loadmat(os.path.join(path_read, f'tempConv_{pname}_sigma{sigma}_2005_2019.mat'))

In [11]:
mat2=matlab.loadmat(os.path.join(f'tempConv_{pname}_sigma{sigma}_2005_2019.mat'))

In [27]:
np.all(np.isclose(mat['data'], mat2['data'], equal_nan=True))

True

In [28]:
np.all(np.isclose(mat['data_conv'], mat2['data_conv'], equal_nan=True))

True